In [1]:
import sys
import numpy as np
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
import cv2
import time

In [ ]:
class MyApp(QWidget):

    def __init__(self):
        super().__init__()

        self.initUI()

    def initUI(self):

        board_cv2 = cv2.imread('image/board.png')
        self.board_cv2 = cv2.cvtColor(board_cv2, cv2.COLOR_BGR2RGB)

        white_ball = cv2.imread('image/white.png')
        self.white_ball = cv2.cvtColor(white_ball, cv2.COLOR_BGR2RGB)

        black_ball = cv2.imread('image/black.png')
        self.black_ball = cv2.cvtColor(black_ball, cv2.COLOR_BGR2RGB)

        # numpy to QImage
        height, width, channel = self.board_cv2.shape
        bytesPerLine = 3 * width
        qImg_board = QImage(self.board_cv2.data, width, height, bytesPerLine, QImage.Format_RGB888)
        
        self.lbl_img = QLabel()
        self.lbl_img.setPixmap(QPixmap(qImg_board))

        self.vbox = QVBoxLayout()
        self.vbox.addWidget(self.lbl_img)
        self.setLayout(self.vbox)
        
        self.setWindowTitle('lakiaro')
        self.move(100, 100)
        self.resize(500,500)
        self.show()
        
    def game_play(self, board_img, ball, pos_x, pos_y, turn):
        #human

        ball_size = ball.shape[0]
        step_size = 56
        off_set = 10

        # 판의 마지막 모서리에는 돌을 두지 못하게 한다.
        if pos_x < step_size/2+off_set+1 or pos_y < step_size/2+off_set+1:
            print('그곳에는 둘 수 없습니다')

        elif pos_x > step_size*self.board_size+step_size/2+off_set or pos_y > step_size*self.board_size+step_size/2+off_set:
            print('그곳에는 둘 수 없습니다')

        else:

            step_x = round((pos_x - off_set)/step_size)
            step_y = round((pos_y - off_set)/step_size)

            if self.board[step_x-1,step_y-1] != 0: # 이미 돌이 있을때
                print('그곳에는 둘 수 없습니다')

            else:
                self.board[step_x-1,step_y-1] = turn
                self.board_history[step_x-1,step_y-1] = self.cnt
                self.cnt = self.cnt + 1
                
                x_step = step_size*step_x-round(step_size/2) + off_set
                y_step = step_size*step_y-round(step_size/2) + off_set
                
                board_img[x_step:x_step+ball_size,y_step:y_step+ball_size] = ball

                # 게임 결과 확인
                if self.game_rule(self.board, turn):
                    self.game_end = 1

                    print('게임이 끝났습니다.')

                    board_img = cv2.cvtColor(board_img, cv2.COLOR_RGB2BGR)
                    

                    print('축하합니다 당신이 승리 하였습니다')


                                  
        return board_img
    
#     def mousePressEvent(self, event):
#         if event.buttons() & Qt.LeftButton:
#             print('LEFT')
#         if event.buttons() & Qt.MidButton:
#             print('MIDDLE')
#         if event.buttons() & Qt.RightButton:
#             print('RIGHT')
    def mousePressEvent(self, e):

        x = e.x()
        y = e.y()
        if self.game_end == 0:

            # 흑돌 사람 게임 플레이

            self.board_cv2 = self.game_play(self.board_cv2, self.black_ball, y, x, 1)
           
            save_name =  'result/' + str(self.cnt) + "board_black.png"
            save_name_w = 'result/' + str(self.cnt) + "board_white.png"
            save_name_pred = 'result/' + str(self.cnt) + "board_pred.png"

            

            # 백돌 인공지능 게임 플레이
            input_X = self.board.flatten()/2
            result = self.sess.run(self.logits, feed_dict={self.X: input_X[None,:]})
            result_mat = self.sigmoid(result).reshape([self.board_size,self.board_size])

            heat_map = cv2.resize(result_mat*255, (500, 500))            
            result_mat, max_x, max_y = self.find_max(result_mat)

            save_image = cv2.resize(self.board_cv2, (500, 500), interpolation=cv2.INTER_CUBIC)
            save_image = cv2.cvtColor(save_image, cv2.COLOR_RGB2BGR)
            #save_image[:,:,0] = save_image[:,:,0] + heat_map

            cv2.imwrite(save_name, save_image)
            cv2.imwrite(save_name_pred, heat_map)


            # 인공지능이 판단한 최적의 위치가 만약 둘수 없는 곳이면 다시 계산
            while self.board[max_x,max_y] !=0:
                print('AI 거긴 둘 수 없다')
                result_mat, max_x, max_y = self.find_max(result_mat)


            self.board[max_x,max_y] = 2 # 인공지능은 항상 2값 / 사람은 1값으로 표현
            self.board_history[max_x,max_y] = self.cnt
            self.cnt = self.cnt + 1

            ball_size = self.white_ball.shape[0]
            step_size = 56
            off_set = 10
            x_step = step_size*(max_x+1)-round(step_size/2) + off_set
            y_step = step_size*(max_y+1)-round(step_size/2) + off_set
            self.board_cv2[x_step:x_step+ball_size,y_step:y_step+ball_size] = self.white_ball

             
            save_image = cv2.resize(self.board_cv2, (500, 500), interpolation=cv2.INTER_CUBIC)
            save_image = cv2.cvtColor(save_image, cv2.COLOR_RGB2BGR)   
            cv2.imwrite(save_name_w, save_image)
            #self.board_history[step_x-1,step_y-1] = self.cnt
            
            #self.board_cv2 = self.game_play(self.board_cv2, self.white_ball, y, x, 2)
                
           
            height, width, channel = self.board_cv2.shape
            bytesPerLine = 3 * width
            qImg_board = QImage(self.board_cv2.data, width, height, bytesPerLine, QImage.Format_RGB888)
            self.lbl_img.setPixmap(QPixmap(qImg_board))


        
if __name__ == '__main__':

    app = QApplication(sys.argv)
    ex = MyApp()
    sys.exit(app.exec_())